# Add unique identifiers to your tables

Generate UUIDs for automatic row identification.

## Problem

You need unique identifiers for rows in your data pipeline. Maybe you're building an API that returns specific records, tracking processing status across systems, or joining data from multiple sources.

| Use case | Why UUIDs help |
|----------|----------------|
| API responses | Return stable IDs clients can reference |
| Distributed systems | Generate IDs without coordination |
| Data lineage | Track records across pipeline stages |

## Solution

**What's in this recipe:**

- Create tables with auto-generated UUID primary keys
- Add UUID columns to existing tables
- Generate UUIDs with `uuid7()`

You use `uuid7()` to generate UUIDs for each row. Define it in the schema with `{'column_name': uuid7()}` syntax, or add it to existing tables with `add_computed_column()`.

### Setup

In [ ]:
%pip install -qU pixeltable

In [1]:
import pixeltable as pxt
from pixeltable.functions.uuid import uuid7

In [ ]:
# Create a fresh directory
pxt.drop_dir('uuid_demo', force=True)
pxt.create_dir('uuid_demo')

### Create a table with a UUID primary key

Use `uuid7()` in your schema to create a column that auto-generates UUIDs:

In [3]:
# Create table with auto-generated UUID primary key
products = pxt.create_table(
    'uuid_demo/products',
    {
        'id': uuid7(),  # Auto-generates UUID for each row
        'name': pxt.String,
        'price': pxt.Float,
    },
    primary_key=['id'],
)

Created table 'products'.


In [4]:
# Insert data - no need to provide 'id', it's auto-generated
products.insert(
    [
        {'name': 'Laptop', 'price': 999.99},
        {'name': 'Mouse', 'price': 29.99},
        {'name': 'Keyboard', 'price': 79.99},
    ]
)

Inserted 3 rows with 0 errors in 0.02 s (191.21 rows/s)


3 rows inserted.

In [5]:
# View the data - each row has a unique UUID
products.collect()

id,name,price
019c00eb-b464-7170-8fc8-d96bddf56508,Laptop,999.99
019c00eb-b464-7170-8fc8-d96c137f00f4,Mouse,29.99
019c00eb-b464-7170-8fc8-d96dfaad1c2a,Keyboard,79.99


### Add a UUID column to an existing table

You can add a UUID column to a table that already exists using `add_computed_column()`:

In [6]:
# Create a table without a UUID column
orders = pxt.create_table(
    'uuid_demo/orders', {'customer': pxt.String, 'amount': pxt.Float}
)

Created table 'orders'.


In [7]:
# Insert some data
orders.insert(
    [
        {'customer': 'Alice', 'amount': 150.00},
        {'customer': 'Bob', 'amount': 75.50},
    ]
)

Inserted 2 rows with 0 errors in 0.01 s (310.49 rows/s)


2 rows inserted.

In [8]:
# Add a UUID column to existing table
orders.add_computed_column(order_id=uuid7())

Added 2 column values with 0 errors in 0.02 s (98.14 rows/s)


2 rows updated.

In [9]:
# View orders with their UUID column
orders.collect()

customer,amount,order_id
Alice,150.,019c00eb-e605-7112-81bd-90a8e8e40943
Bob,75.5,019c00eb-e605-7112-81bd-90a9b86c26af


## Explanation

**Two ways to add UUIDs:**

| Approach | Use case |
|----------|----------|
| `uuid7()` in schema | Auto-generated UUID at table creation |
| `add_computed_column(col=uuid7())` | Add UUID column to existing table |

Both use `uuid7()` which generates UUIDv7 (time-based) identifiers:

- 128-bit values
- Formatted as 32 hex digits with hyphens: `018e65c5-35e5-7c5d-8f37-f1c5b9c8a7b2`
- Time-ordered for better database performance
- Virtually guaranteed unique (collision probability is negligible)

## See also

- [Tables and operations](https://docs.pixeltable.com/tutorials/tables-and-data-operations)
- [Computed columns](https://docs.pixeltable.com/tutorials/computed-columns)